**Introduction**

This project notebook performs very rudimentary exploratory data analysis and visualization of a coffee quality score dataset. The original data was derived from the Coffee Quality Institute (CQI), downloaded from Kaggle.com (https://www.kaggle.com/datasets/volpatto/coffee-quality-database-from-cqi). The data visualization and filtering widgets were created using plotly.express and streamlit for a web application to be deployed via Render.

The dataset contains just over 1,300 entries of Arabica bean varieties from all over the world. Most of the meta data that was beyond the scope of this project was removed during the data cleaning. Some of the farm meta data was preserved for informational purposes.

In general, coffee scores are calculated based on a number of criteria such as aroma, acidity, and flavor. Additionally, points are deducted for defects in the coffee bean sample. Overall score is out of 100 and is assigned into one of the following ratings:
- 65-79: Commodity coffee.
- 80-89: Specialty coffee
- 90+: Presidential Award

In [1]:
# Module imports
import pandas as pd
import plotly.express as px
import streamlit as st
import re

In [2]:
# Data reading and options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 0)
df_coffee = pd.read_csv('arabica_data_cleaned.csv')
df_coffee.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1311 entries, 0 to 1310
Data columns (total 44 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             1311 non-null   int64  
 1   Species                1311 non-null   object 
 2   Owner                  1304 non-null   object 
 3   Country.of.Origin      1310 non-null   object 
 4   Farm.Name              955 non-null    object 
 5   Lot.Number             270 non-null    object 
 6   Mill                   1001 non-null   object 
 7   ICO.Number             1165 non-null   object 
 8   Company                1102 non-null   object 
 9   Altitude               1088 non-null   object 
 10  Region                 1254 non-null   object 
 11  Producer               1081 non-null   object 
 12  Number.of.Bags         1311 non-null   int64  
 13  Bag.Weight             1311 non-null   object 
 14  In.Country.Partner     1311 non-null   object 
 15  Harv

**Data Cleaning**

As previously mentioned, columns containing information that was out of this project's scope were filtered out. Columns were renamed to snake_case, and empty values were replaced with 'unknown'. Of note, the only non-categorical column with empty values was harvest year. This column had multiple formats for the value, including a range of years. Since no data analysis was performed on harvest year, empty values were still replaced by 'unknown' and the data type remained unchanged.

Lastly, checked and removed any duplicates and partial data (e.g. valid entries with no score).

In [3]:
df_coffee.rename(str.lower, axis=1, inplace=True)
df_coffee.columns = [re.sub('[\.\s]', '_', col) for col in df_coffee]

cols_to_filter = ['unnamed:_0',
                'owner',
                'lot_number',
                'mill',
                'ico_number',
                'altitude',
                'producer',
                'grading_date',
                'owner_1',
                'expiration',
                'certification_body',
                'certification_address',
                'certification_contact',
                'unit_of_measurement',
                'altitude_low_meters',
                'altitude_high_meters',
                'altitude_mean_meters']

df_arabica = df_coffee.loc[df_coffee['total_cup_points'] > 0, ~df_coffee.columns.isin(cols_to_filter)]
df_arabica.fillna('unknown', inplace=True)

print(df_arabica.duplicated().sum())

# No duplicates to remove. Checking general info to verify changes.
df_arabica.info()

0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1310 entries, 0 to 1309
Data columns (total 27 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   species               1310 non-null   object 
 1   country_of_origin     1310 non-null   object 
 2   farm_name             1310 non-null   object 
 3   company               1310 non-null   object 
 4   region                1310 non-null   object 
 5   number_of_bags        1310 non-null   int64  
 6   bag_weight            1310 non-null   object 
 7   in_country_partner    1310 non-null   object 
 8   harvest_year          1310 non-null   object 
 9   variety               1310 non-null   object 
 10  processing_method     1310 non-null   object 
 11  aroma                 1310 non-null   float64
 12  flavor                1310 non-null   float64
 13  aftertaste            1310 non-null   float64
 14  acidity               1310 non-null   float64
 15  body               

**Data Analysis and Visualization**

The following sections will analyze coffee bean score by variety. Specifically, we'll look at:
- Distribution of total cup points by bean variety
- A scatterplot comparing individual categories by bean variety
- Distribtion of total cup points by country

To facilitate some of our calculations, we can store the categories on which each bean is rated into a list called categories.

In [4]:
categories = ['aroma', 'flavor', 'aftertaste', 'acidity', 'body', 'balance', 'uniformity', 'clean_cup', 'sweetness']

Since we're primarily concerned with coffee scores, we can look at the sample size of each variety along with aggregate data within each score category. We can sort the results by count to see which variety is the most popular in the dataset.

In [5]:
display(df_arabica.groupby('variety')[categories].agg(('count', 'min', 'max', 'mean')).sort_values([('aroma', 'count')], ascending=False))


aroma                       flavor              \
                      count   min   max      mean  count   min   max   
variety                                                                
Caturra                255   5.08  8.25  7.606314  255    6.50  8.33   
Bourbon                226   6.17  8.50  7.557301  226    6.08  8.50   
Typica                 211   6.67  8.17  7.474502  211    6.33  8.17   
unknown                201   6.75  8.67  7.617960  201    6.50  8.83   
Other                  108   7.08  8.75  7.623704  108    6.50  8.67   
Catuai                 74    6.67  8.50  7.491892  74     6.17  8.00   
Hawaiian Kona          44    6.92  8.08  7.513182  44     6.92  8.17   
Yellow Bourbon         35    6.92  8.42  7.498000  35     7.00  8.00   
Mundo Novo             33    6.75  8.00  7.482424  33     6.92  8.00   
Catimor                20    7.42  8.42  7.680000  20     7.25  8.25   
SL14                   17    7.33  8.42  7.882353  17     6.92  8.17   
SL28                   15    7.17  8.08  7.744667  15     7.08  8.17   
Pacas                  13    6.83  7.75  7.396923  13     6.83  7.83   
Gesha                  12    7.33  8.00  7.624167  12     7.17  8.00   
Pacamara               8     7.33  7.92  7.655000  8      7.25  8.17   
SL34                   8     7.00  8.08  7.771250  8      7.08  8.08   
Arusha                 5     7.08  7.75  7.568000  5      7.33  7.58   
Peaberry               5     7.17  7.83  7.616000  5      7.00  7.83   
Sumatra                3     7.17  8.00  7.473333  3      7.42  8.00   
Mandheling             3     7.58  7.81  7.686667  3      7.42  7.88   
Ruiru 11               2     7.67  7.75  7.710000  2      7.25  7.67   
Java                   2     6.92  7.92  7.420000  2      6.83  7.83   
Blue Mountain          2     7.42  7.75  7.585000  2      7.33  7.58   
Ethiopian Yirgacheffe  2     8.08  8.17  8.125000  2      7.83  8.17   
Marigojipe             1     7.33  7.33  7.330000  1      7.50  7.50   
Pache Comun            1     7.42  7.42  7.420000  1      7.17  7.17   
Sulawesi               1     7.50  7.50  7.500000  1      7.58  7.58   
Ethiopian Heirlooms    1     7.58  7.58  7.580000  1      7.58  7.58   
Sumatra Lintong        1     7.75  7.75  7.750000  1      7.83  7.83   
Moka Peaberry          1     7.08  7.08  7.080000  1      7.50  7.50   

                                aftertaste                       acidity  \
                           mean      count   min   max      mean   count   
variety                                                                    
Caturra                7.559882  255        6.33  8.08  7.467255  255      
Bourbon                7.498540  226        6.17  8.42  7.315664  226      
Typica                 7.380995  211        6.17  8.00  7.281090  211      
unknown                7.612090  201        6.42  8.67  7.499701  201      
Other                  7.569444  108        6.50  8.58  7.455185  108      
Catuai                 7.425405  74         6.17  8.00  7.306081  74       
Hawaiian Kona          7.531591  44         6.83  8.00  7.467045  44       
Yellow Bourbon         7.540000  35         6.83  8.00  7.404857  35       
Mundo Novo             7.492121  33         6.67  7.83  7.340909  33       
Catimor                7.650500  20         7.08  8.08  7.500000  20       
SL14                   7.717059  17         7.17  8.17  7.594118  17       
SL28                   7.732667  15         7.42  8.00  7.689333  15       
Pacas                  7.410769  13         6.67  7.67  7.232308  13       
Gesha                  7.535833  12         6.92  7.92  7.375000  12       
Pacamara               7.688750  8          6.92  7.83  7.460000  8        
SL34                   7.790000  8          7.00  8.00  7.666250  8        
Arusha                 7.498000  5          7.17  7.50  7.368000  5        
Peaberry               7.366000  5          7.00  7.83  7.332000  5        
Sumatra                7.696667  3          7.33 

We can see from the previous results that the most popular Arabica bean varieties are Caturra, Bourbon, and Typica. We can determine how these popular varieties are rated based on their scores in this dataset.

In [6]:
popular_var = ['Caturra', 'Bourbon', 'Typica']
df_popular_var = df_arabica.query('variety in @popular_var')

display(df_popular_var.groupby('variety')['total_cup_points'].describe())

,count,mean,std,min,25%,50%,75%,max
variety,,,,,,,,
Bourbon,226.0,81.946903,2.536126,68.33,81.00,82.33,83.3975,89.75
Caturra,255.0,82.762275,2.114854,63.08,82.00,83.17,83.7900,87.17
Typica,211.0,81.020758,2.588114,67.92,79.79,81.50,82.6700,85.33


Caturra's minimum value is 0 (which is likely due to an incomplete data entry), but the sample sizes and aggregate score values are similar enough to conclude that all the samples within these varieties range from commodity coffee to speciality coffee. None of these samples, however, are rated in the highest tier.

We can plot the distribution of total scores among all the varieties to see how these popular bean types compare to the others. We'll set a minimum score of 65 as this is lowest bracket for specialty coffee breakdowns (and will serve as a placeholder for a slider widget in the web app).

In [7]:
min_score = 65
df_score = df_arabica[df_arabica['total_cup_points'] >= min_score]
fig = px.histogram(df_score, x='total_cup_points', color='variety')
fig.update_layout(xaxis_title='Score out of 100', yaxis_title='Number of Samples')

fig.show()

Even though individual scores contribute to the overall score, we can plot individual categories against each other to see which varieties offer up high scores for two criteria. For example, if we're primarily concerned with a bean's aroma and flavor, we can see if there's a correlation between the two criteria within the popular varieties.

In [8]:
# Place holders for steamlit widgets
cat1 = 'flavor'
cat2 = 'aroma'

fig = px.scatter(df_popular_var, x=cat1, y=cat2, color='variety')
fig.show()

Now that we have an idea of coffee scores by variety, we can compare coffee scores by country to determine where the highest rated coffee beans originate and which varieties they produce. Rather than taking the highest scores alone, we'll explore which countries produce beans with at least a specialty rating of 80. We will alos omit bean varieties that are unknown, or other.

In [9]:
variety_filter = (df_arabica['variety'].str.lower() != 'unknown') & (df_arabica['variety'].str.lower() != 'other')
df_arabica_filtered = df_arabica[variety_filter]

df_countries = df_arabica_filtered.groupby('country_of_origin')['total_cup_points'].min().sort_values(ascending=False).reset_index()
top_countries = df_countries[df_countries['total_cup_points'] >= 80]
df_top_countries = df_arabica_filtered.query('country_of_origin in @top_countries.country_of_origin')

display(df_top_countries.groupby('country_of_origin')['variety'].nunique())

country_of_origin
Burundi                         1
Ethiopia                        5
Japan                           1
Laos                            1
Malawi                          2
Myanmar                         1
Panama                          3
Philippines                     1
Rwanda                          1
Tanzania, United Republic Of    3
Uganda                          2
United States                   1
Vietnam                         2
Zambia                          1
Name: variety, dtype: int64

Now that we see Ethopia produces the most varieties of specialty coffees, we can explore which varieties from Ethopia are rated the highest.

In [10]:
df_ethopia = df_arabica_filtered[df_arabica_filtered['country_of_origin'].str.lower() == 'ethiopia']
display(df_ethopia[['variety', 'total_cup_points']])

,variety,total_cup_points
19,Ethiopian Yirgacheffe,87.25
59,Caturra,85.75
82,Typica,85.33
131,Ethiopian Yirgacheffe,84.67
220,Gesha,84.17
241,Gesha,84.00
537,Ethiopian Heirlooms,82.92


While Ethiopia produces two of the most popular varieties (Caturra and Typica), they also produce other varieties of great coffee!

In the web app verison of this project, the user will be able to select two different countries to compare their distributions of coffee scores. For this notebook, we'll compare Ethiopia coffee with another popular coffee producing country, Guatemala.

In [11]:
# Place holders for steamlit widgets
country_1 = 'Ethiopia'
country_2 = 'Guatemala'

country_filter = (df_arabica['country_of_origin'] == country_1) | (df_arabica['country_of_origin'] == country_2)
df_filtered = df_arabica[country_filter]

fig = px.histogram(df_filtered, x='total_cup_points', color='country_of_origin', barmode='overlay')
fig.update_layout(xaxis_title='Score out of 100', yaxis_title='Count')
fig.show()

The two countries show a fairly normal distribution of scores, with Guatemala appearing to have a higher volume of samples. In terms of rating, the majority of coffee beans produced in each country are specialty coffees, with Ethiopia generally having a slightly higher score. Both samples also show a rating that is at or on the cusp of the highest rating!

**Conclusion**

Arabica (and Robusta) beans are the most popular coffee bean in the world, but there are many varieties within the species each with a different rating. According to the dataset, the most popular varieties are Bourbon, Typica, and Caturra. These bean varieties, for the most part, are rated as specialty coffees, but almost none of them score in the highest tier. Ethiopia appears to produce the most variety of specialty coffees, including a variety (although unknown) in the highest tier.